In [16]:
from model import AttentionModel

In [10]:
keyPoints = {
    'NOSE': 0,
    'LEFT_EYE': 1,
    'RIGHT_EYE': 2,
    'LEFT_EAR':3,
    'RIGHT_EAR': 4,
    'LEFT_SHOULDER': 5,
    'RIGHT_SHOULDER':6,
    'LEFT_ELBOW': 7,
    'RIGHT_ELBOW': 8,
    'LEFT_WRIST': 9,
    'RIGHT_WRIST': 10,
    'LEFT_HIP': 11,
    'RIGHT_HIP': 12,
    'LEFT_KNEE': 13,
    'RIGHT_KNEE': 14,
    'LEFT_ANKLE': 15,
    'RIGHT_ANKLE': 16,
}

In [17]:
MODEL_SLUG = '60_yolov8m_best'

In [18]:
OUT_PATH = f'result/model_output_video_{MODEL_SLUG}.avi' # your video path for saving
DATA_PATH = 'Анализ бригад (телефон)/Есть телефон/00_26_30.mp4' # your video path for processing
SAVE_IMG_PATH = 'result'

In [19]:
model = AttentionModel(f'models/{MODEL_SLUG}.pt')
model.load_models()

In [20]:
model.detected_data

,Frame_id,Time,Position


In [ ]:
model.process_video(DATA_PATH, OUT_PATH, pos_estimation=True, detection=True, save=True)

 26%|██▋       | 1375/5196 [02:06<05:42, 11.14it/s]

In [24]:
import pandas as pd

pd.DataFrame(model.pos_est_values)

,0,1,2,3
0,1,0.083333,2,"[[[1002.46533203125, 663.6934814453125], [989...."
1,2,0.166667,2,"[[[1013.07177734375, 678.38623046875], [996.08..."
2,3,0.250000,2,"[[[1033.189697265625, 691.5792236328125], [101..."
3,4,0.333333,2,"[[[1020.4559326171875, 678.0460205078125], [10..."
4,5,0.416667,2,"[[[1015.8204345703125, 672.3037719726562], [99..."
5,6,0.500000,2,"[[[1021.3914794921875, 670.3060302734375], [10..."
6,7,0.583333,2,"[[[1022.5838623046875, 673.1045532226562], [10..."
7,8,0.666667,2,"[[[1022.0577392578125, 676.2551879882812], [10..."
8,9,0.750000,2,"[[[1028.499267578125, 688.089599609375], [1007..."
9,10,0.833333,3,"[[[1028.941650390625, 692.9158935546875], [100..."


In [ ]:
import pandas as pd

pos_est_data = pd.DataFrame(model.pos_est_values, columns=['Frame_id', 'Time', *model.keypoints])
pos_est_data

In [ ]:
pos_est_data

In [ ]:
model.pos_est_data.head(10)

In [ ]:
model.pos_est_data.info()

In [ ]:
from ultralytics import YOLO

model = YOLO('models/yolov8x-pose.pt').to('cuda')
result = model.predict(DATA_PATH, stream=True, save=True)

In [ ]:
model('result/model_output_video.avi')

In [ ]:
model.pos_est_data.to_csv('result/pos_est_data.csv', index=0)

In [ ]:
model.detected_data.to_csv('result/detected_data.csv', index=0)